In [6]:
%%capture
# STARTER CODE - RUN THIS CELL - DO NOT CHANGE
!pip install category_encoders
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, accuracy_score, recall_score, precision_score
df_r = sns.load_dataset("tips").dropna()
df_c = sns.load_dataset("titanic").dropna()
model_r = LinearRegression()
model_c = LogisticRegression()
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(df_r.drop("total_bill", axis=1), df_r["total_bill"])
x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df_c.drop(["survived", "alive", "adult_male"], axis=1), df_c["survived"])
pipe_r = make_pipeline(ce.OrdinalEncoder(), StandardScaler(), LinearRegression()).fit(x_train_r, y_train_r)
pipe_c = make_pipeline(ce.OrdinalEncoder(), StandardScaler(), LogisticRegression()).fit(x_train_c, y_train_c)
y_pred_r = pipe_r.predict(x_test_r)
y_pred_c = pipe_c.predict(x_test_c)

### Separate your data set into training and testing. (80/20 split)

In [7]:
df = pd.read_csv("seattle_weather_1948-2017.csv").dropna()
df["RAIN"] = df["RAIN"].astype(bool)

In [8]:
training, test = train_test_split(
    df,
    train_size = 0.8,
    test_size = 0.2,
    random_state = 60
)

In [9]:
training["RAIN"].value_counts(normalize=True)

False    0.574567
True     0.425433
Name: RAIN, dtype: float64

In [10]:
test["RAIN"].value_counts(normalize=True)

False    0.568493
True     0.431507
Name: RAIN, dtype: float64

In [11]:
# Create Validation Set
# Select target column
target = "RAIN"

# Create training and validation sets
X_train, X_val, y_train, y_val,= train_test_split(
    training.drop([target, "DATE"], axis=1), # Df with features
    training[target], # Df with labels
    train_size=0.8,
    test_size=0.2,
    random_state=60
)

### Calculate the Precision and Recall for the classification heuristic you made.

In [14]:
def heuristic(df):
    """
    Simple heuristic:
    
    If it rained the past two days then predict rain else predict no rain
    
    Frist two rows are predicted false be default
    """
    
    preds = []
    
    for x in range(len(df)):

        if x<31:
            preds.append(False)
        else:

            if (df.iloc[x-1]["TMIN"] <= 30) | (df.iloc[x-7]["TMIN"] <= 30):
                preds.append(True)
            else:
                preds.append(False)
    return preds
    
df["preds"] = heuristic(df)

FP = np.zeros(len(df))
TP = np.zeros(len(df))
FN = np.zeros(len(df))
TN = np.zeros(len(df))

for x in range(len(df)):
    if (df["TMIN"].iloc[x] <= 30) & (df["preds"].iloc[x] == True):
        TP[x] = 1
        
    elif (df["RAIN"].iloc[x] > 30) & (df["preds"].iloc[x] == False):
        TN[x] = 1
        
    elif (df["RAIN"].iloc[x] <= 30) & (df["preds"].iloc[x] == False):
        FN[x] = 1

    else:
        FP[x] = 1

In [15]:
# Create A Function to Calculate Precision
def precision(TP, FP):
    return TP.sum()/(TP+FP).sum()
precision(TP, FP)

0.3866935483870968

In [16]:
# Create A Function to Calculate Recall
def recall(TP, FN):
    return TP.sum()/(TP+FN).sum()
recall(TP, FN)

0.03991343072376909

In [17]:
# Create a Function to Calculate Accuracy
def accuracy(TP, FP, TN, FN):
    return (TP+TN).sum()/(TP+TN+FP+FN).sum()

accuracy(TP, FP, TN, FN)

0.037537184906842026

### Calculate the MSE, MAE, or SSE for the regression heuristic

In [18]:
# Create a Mean Squared Error Function
def mse(y_true, y_pred):
    return np.square(y_true - y_pred).mean()
mse(y_test_r, y_pred_r)

41.11033586496887

In [19]:
# Your Mean Squared Error function should equal this value
mean_squared_error(y_test_r, y_pred_r)

41.11033586496887

In [20]:
# Create a Mean Absolute Error Function
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
mae(y_test_r, y_pred_r)

4.9236334705691105

In [21]:
# Your Mean Absolute Error function should equal this value
mean_absolute_error(y_test_r, y_pred_r)

4.923633470569111

In [22]:
# Create a Sum Squared Error Function
def sse(y_true, y_pred):
    return np.square(y_true - y_pred).sum()
sse(y_test_r, y_pred_r)

2507.730487763101

###  Repeat the process 5 times.

In [23]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

lr.predict

lr.score(X_val, y_val)

0.925880626223092

In [24]:
# Look at classification report

# store predictions
preds = lr.predict(X_val)

print(classification_report(y_val, preds))

              precision    recall  f1-score   support

       False       0.89      0.99      0.94      2313
        True       0.99      0.84      0.91      1775

    accuracy                           0.93      4088
   macro avg       0.94      0.92      0.92      4088
weighted avg       0.93      0.93      0.92      4088



In [25]:
scores = cross_val_score(lr,X_val, y_val, cv=5)

scores.mean()

0.891633473289182